In [140]:
import pandas as pd
import datetime as dt
import numpy as np
from google.colab                import drive

In [141]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [142]:
df_segmentacao = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/Segmentação de Clientes/online_retail_II.xlsx').drop('Description', axis=1)
df_segmentacao.shape

(525461, 7)

In [143]:
#Retiramos dados nulos de consumidores
df_segmentacao = df_segmentacao[~df_segmentacao['Customer ID'].isna()]

In [144]:
#Verificamos que existem quantidades negativas de compras
df_segmentacao = df_segmentacao[~df_segmentacao['Quantity'] < 1]

In [145]:
#Retiramos dados de clientes que tiveram compra com valor nulo
df_segmentacao = df_segmentacao[df_segmentacao['Price'] != 0]

In [146]:
#Intervalo de tempo da base de dados
df_segmentacao['InvoiceDate'].agg(['min', 'max'])

#A data máxima será a medida final do intervalo de recencia
data_máxima = max(df_segmentacao['InvoiceDate'])

In [147]:
df_segmentacao['Customer ID'] = df_segmentacao['Customer ID'].astype(int).astype(str)
df_segmentacao.shape

(411981, 7)

In [148]:
#Quantidade de clientes únicos pertencentes a base de dados
len(df_segmentacao['Customer ID'].unique().tolist())

4362

In [149]:
df_segmentacao.dtypes

Invoice                object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID            object
Country                object
dtype: object

In [150]:
#Criar análise de RFM para a base de clientes
df_segmentacao['Amount'] = df_segmentacao['Price'] * df_segmentacao['Quantity']

In [151]:
df_segmentacao.columns

Index(['Invoice', 'StockCode', 'Quantity', 'InvoiceDate', 'Price',
       'Customer ID', 'Country', 'Amount'],
      dtype='object')

In [152]:
#Cria feature de montante transacionado
df_monetary = df_segmentacao.groupby('Customer ID')['Amount'].sum().reset_index()

#Cria feature de invoice consumida
df_frequency = df_segmentacao.groupby('Customer ID')['Invoice'].count().reset_index()


#Junta as features tratadas
rfm_dataframe = df_monetary.merge(df_frequency, on='Customer ID')

In [153]:
#Trazer feature de recência de cada consumidor
df_segmentacao['Recency'] = data_máxima - df_segmentacao['InvoiceDate']

#Buscamos a data mínima em dias de cada cliente
df_recency = df_segmentacao.groupby('Customer ID')['Recency'].min().reset_index()

#Tratamos a data
df_recency['Recency'] = df_recency['Recency'].dt.days

In [154]:
rfm_dataframe = rfm_dataframe.merge(df_recency, on='Customer ID')

In [155]:
rfm_dataframe.columns

Index(['Customer ID', 'Amount', 'Invoice', 'Recency'], dtype='object')

In [156]:
#Quantidade de consumidores únicos
len(rfm_dataframe['Customer ID'].unique().tolist())

4362

In [157]:
# removing outliers
# outlier treatment for amount
Q1 = rfm_dataframe.Amount.quantile(0.05)
Q3 = rfm_dataframe.Amount.quantile(0.95)
IQR = Q3 - Q1
rfm_dataframe = rfm_dataframe[(rfm_dataframe.Amount >= Q1 - 1.5*IQR) & (rfm_dataframe.Amount <= Q3 + 1.5*IQR)]

# outlier treatment for recency
Q1 = rfm_dataframe.Recency.quantile(0.05)
Q3 = rfm_dataframe.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm_dataframe = rfm_dataframe[(rfm_dataframe.Recency >= Q1 - 1.5*IQR) & (rfm_dataframe.Recency <= Q3 + 1.5*IQR)]

# outlier treatment for frequency
Q1 = rfm_dataframe.Invoice.quantile(0.05)
Q3 = rfm_dataframe.Invoice.quantile(0.95)
IQR = Q3 - Q1
rfm_dataframe = rfm_dataframe[(rfm_dataframe.Invoice >= Q1 - 1.5*IQR) & (rfm_dataframe.Invoice <= Q3 + 1.5*IQR)]

In [158]:
len(rfm_dataframe['Customer ID'].unique().tolist())

4277

In [159]:
rfm_dataframe.head()

,Customer ID,Amount,Invoice,Recency
0,12346,27.02,44,66
1,12347,1323.32,71,2
2,12348,222.16,20,73
3,12349,2646.99,107,42
4,12351,300.93,21,10
